In [2]:
import pandas as pd
import numpy as np
import re

from CAD.config import Config

In [3]:
config = Config()
df = pd.read_csv(config.INTERIM_DATA_DIR / 'Merged_Data.csv')

/var/folders/g7/qvkgm1k12kzfvx8hv5_jqcph0000gn/T/ipykernel_59533/2879187287.py:2: DtypeWarning: Columns (9,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(config.INTERIM_DATA_DIR / 'Merged_Data.csv')


In [4]:
df = df.drop('CURR_GRADE_ORD', axis=1)
df['SCHOOL_YEAR'] = df['SCHOOL_YEAR'].str.split('-').str[0]
df['STUDENT_GRADE_LEVEL'] = df['STUDENT_GRADE_LEVEL'].replace(['PK', 'KF', 'OS'], [-1, 0, -2])
df['ECONOMIC_CODE'] = df['ECONOMIC_CODE'].replace(['F', 'N', 'R'], [1, 0, 1])
df['SPECIAL_ED_CODE'] = df['SPECIAL_ED_CODE'].replace(['Y', 'N'], [1, 0])
df['ENG_PROF_CODE'] = df['ENG_PROF_CODE'].replace('NT', -1)
df['HISPANIC_IND'] = df['HISPANIC_IND'].replace(['No', 'Yes'], [0, 1])

/var/folders/g7/qvkgm1k12kzfvx8hv5_jqcph0000gn/T/ipykernel_59533/2389374588.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['ECONOMIC_CODE'] = df['ECONOMIC_CODE'].replace(['F', 'N', 'R'], [1, 0, 1])
/var/folders/g7/qvkgm1k12kzfvx8hv5_jqcph0000gn/T/ipykernel_59533/2389374588.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['SPECIAL_ED_CODE'] = df['SPECIAL_ED_CODE'].replace(['Y', 'N'], [1, 0])
/var/folders/g7/qvkgm1k12kzfvx8hv5_jqcph0000gn/T/ipykernel_59533/2389374588.py:7: FutureWarning: Downcasting behavior in `replac

In [5]:
non_numeric_columns = ['SCHOOL_NAME', 'DISTRICT_NAME', 'ETHNIC_CODE', 'STUDENT_GENDER']
float_cols = ['Total_Days_Unexcused_Absent', 'Total_Days_Enrolled', 'Total_Days_Present']

numeric_columns = [col for col in df.columns if col not in [*non_numeric_columns, *float_cols]]

for col in numeric_columns:
    df[col] = df[col].astype(int)

for col in float_cols:
    df[col] = df[col].astype(float)

In [6]:
attendance_cols = ['Total_Days_Present', 'Total_Days_Enrolled', 'Total_Days_Unexcused_Absent']
other_cols = [col for col in df.columns if col not in attendance_cols]

agg_dict = {key: 'sum' for key in attendance_cols}
agg_dict.update({key: 'last' for key in other_cols})
grouped_df = df.groupby(by=['STUDENT_ID', 'SCHOOL_YEAR']).agg(agg_dict).reset_index(drop=True)
df = grouped_df[[*df.columns, *grouped_df.columns.difference(df.columns)]]

In [9]:
# df.loc[1688836, 'Total_Days_Present'] = 54.0
# df.loc[731523, 'Total_Days_Present'] = 1.0
# df.loc[1904340, 'Total_Days_Present'] = 2.0
# df.loc[1904340, 'Total_Days_Unexcused_Absent'] = 8.0
# df.loc[1904340, 'Total_Days_Enrolled'] = 10.0
# df.loc[[1016188, 1227776, 1400304, 1401473, 1413193], 'Total_Days_Unexcused_Absent'] = 0.0
# df.loc[686159, 'Total_Days_Unexcused_Absent'] = 3.5
# df.loc[687907, 'Total_Days_Unexcused_Absent'] = 4.0

In [7]:
#12-th grade students
grade_12_students = df[df['STUDENT_GRADE_LEVEL'] == 12]

# Days Enrolled Greater than Unexcused + Present
days_enr_gt_unexcpluspres = df[(df['Total_Days_Unexcused_Absent'] + df['Total_Days_Present']) > df['Total_Days_Enrolled']]

# Grade inconsistencies (demotion of grades)
df_sorted = df.sort_values(by=['STUDENT_ID', 'SCHOOL_YEAR'])
grade_mask = df_sorted.groupby('STUDENT_ID')['STUDENT_GRADE_LEVEL'].diff().lt(0)
grade_inconsitency_df = df[df['STUDENT_ID'].isin(df[grade_mask]['STUDENT_ID'].values)].sort_values(by='STUDENT_ID')

# Drop list
negative_values_indices = [1688836, 731523, 1904340, *[1016188, 1227776, 1400304, 1401473, 1413193], 686159, 687907]

drop_list = [*days_enr_gt_unexcpluspres.index, *grade_inconsitency_df.index, *grade_12_students.index, *negative_values_indices, *df[df['Total_Days_Enrolled'] > 300].index]

df = df.drop(drop_list, axis=0)

In [8]:
mask = df.groupby('STUDENT_ID')['STUDENT_GENDER'].transform('nunique').gt(1)
gender_change_df = df[mask].sort_values(by=['STUDENT_ID', 'SCHOOL_YEAR'])
gender_change_df['STUDENT_GENDER'] = gender_change_df.groupby('STUDENT_ID')['STUDENT_GENDER'].transform('last')

df = df.drop(gender_change_df.index, axis=0)
df = pd.concat([df, gender_change_df], axis=0).reset_index(drop=True)

df['Attendance_Rate'] = df['Total_Days_Present'] / df['Total_Days_Enrolled']

In [9]:
count_series = df.groupby('STUDENT_ID')['SCHOOL_YEAR'].nunique()
training_data = df[df['STUDENT_ID'].isin(count_series[count_series == 6].index)].sort_values(by=['STUDENT_ID', 'SCHOOL_YEAR'])

In [10]:
five_year_data = df[df['STUDENT_ID'].isin(count_series[count_series == 5].index)].sort_values(by=['STUDENT_ID', 'SCHOOL_YEAR'])
non_imputation_data = five_year_data[five_year_data['STUDENT_GRADE_LEVEL'] == -1]
dropping_data = five_year_data[five_year_data['STUDENT_ID'].isin(non_imputation_data['STUDENT_ID'].unique())]

five_year_data = five_year_data.drop(dropping_data.index, axis=0).reset_index(drop=True)

In [11]:
# Checking if linear interpolation is good

def need_non_linear_impute(group, thresh=0.2):
    g = group.set_index('SCHOOL_YEAR').sort_index()
    missing = set([2019, 2020, 2021, 2022, 2023]) - set(g.index)

    for yr in missing:
        if yr - 1 in g.index and yr + 1 in g.index:
            left, right = int(g.loc[[yr - 1, yr + 1], 'Total_Days_Present'].values[0]), int(g.loc[[yr - 1, yr + 1], 'Total_Days_Present'].values[0])
            if abs(right - left) / max(1, max(left, right)) > thresh:
                return True
        
    return False


five_year_data.groupby('STUDENT_ID').apply(need_non_linear_impute).sum()

/var/folders/g7/qvkgm1k12kzfvx8hv5_jqcph0000gn/T/ipykernel_59533/1568285024.py:16: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  five_year_data.groupby('STUDENT_ID').apply(need_non_linear_impute).sum()


np.int64(0)

In [12]:
# Imputing missing data for five year students using linear interpolation
full_years = range(2019, 2025)
attendance_cols = ['Total_Days_Present', 'Total_Days_Enrolled', 'Total_Days_Unexcused_Absent']
explicit_columns = ['STUDENT_GRADE_LEVEL', 'Attendance_Rate']
static_cols = [col for col in five_year_data.columns if col not in attendance_cols + explicit_columns and col != 'SCHOOL_YEAR']

def impute(group):
    g = group.set_index('SCHOOL_YEAR').sort_index()
    g = g.reindex(full_years)

    g['Is_Imputed'] = g[attendance_cols].isna().any(axis=1)

    g[static_cols] = g[static_cols].ffill().bfill()
    g[attendance_cols] = g[attendance_cols].interpolate(method='linear', limit_direction='both').round(0).astype('float')
    g['Attendance_Rate'] = g['Total_Days_Present'] / group['Total_Days_Enrolled']

    g['STUDENT_GRADE_LEVEL_Check'] = g['STUDENT_GRADE_LEVEL'].ffill().bfill()
    g['STUDENT_GRADE_LEVEL'] = g['STUDENT_GRADE_LEVEL'].fillna(g['STUDENT_GRADE_LEVEL_Check'] + 1)

    return g.reset_index()

five_year_data = five_year_data.groupby('STUDENT_ID').apply(impute)

/var/folders/g7/qvkgm1k12kzfvx8hv5_jqcph0000gn/T/ipykernel_59533/3221770233.py:22: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  five_year_data = five_year_data.groupby('STUDENT_ID').apply(impute)


In [13]:
# Cleaning Up and Rejoining
five_year_data = five_year_data.reset_index(drop=True)
five_year_data = five_year_data.drop(['STUDENT_GRADE_LEVEL_Check'], axis=1)
five_year_data['Attendance_Rate'] = five_year_data['Total_Days_Present'] / five_year_data['Total_Days_Enrolled']

integer_cols = [col for col in five_year_data.select_dtypes(include='number') if col not in attendance_cols + ['Attendance_Rate']]
five_year_data[integer_cols] = five_year_data[integer_cols].astype(int)

training_df = pd.concat([training_data, five_year_data], axis=0).sort_values(by=['STUDENT_ID', 'SCHOOL_YEAR']).reset_index(drop=True)
training_df['Is_Imputed'] = training_df['Is_Imputed'].fillna(0).astype(int)

In [14]:
# Pivoting
attendance_pivot = training_df.pivot(index='STUDENT_ID', columns='SCHOOL_YEAR', values='Attendance_Rate').add_prefix('Attendance_Rate_').reset_index().rename_axis(None, axis=1)

absent_pivot = training_df.pivot(index='STUDENT_ID', columns='SCHOOL_YEAR', values='Total_Days_Unexcused_Absent').add_prefix('Total_Days_Unexcused_Absent_').rename_axis(None, axis=1).reset_index()

present_pivot = training_df.pivot(index='STUDENT_ID', columns='SCHOOL_YEAR', values='Total_Days_Present').add_prefix('Total_Days_Present_').rename_axis(None, axis=1).reset_index()

enrolled_pivot = training_df.pivot(index='STUDENT_ID', columns='SCHOOL_YEAR', values='Total_Days_Enrolled').add_prefix('Total_Days_Enrolled_').rename_axis(None, axis=1).reset_index()

grade_pivot = training_df.pivot(index='STUDENT_ID', columns='SCHOOL_YEAR', values='STUDENT_GRADE_LEVEL').add_prefix('STUDENT_GRADE_LEVEL_').rename_axis(None, axis=1).reset_index()

imputed_pivot = training_df.pivot(index='STUDENT_ID', columns='SCHOOL_YEAR', values='Is_Imputed').add_prefix('Is_Imputed_').rename_axis(None, axis=1).reset_index()

In [15]:
non_aggregate_cols = ['Total_Days_Unexcused_Absent', 'Total_Days_Enrolled', 'Total_Days_Present', 'STUDENT_GRADE_LEVEL', 'Attendance_Rate', 'Is_Imputed']

aggreagte_cols = [col for col in training_df.columns if col not in non_aggregate_cols]
last_agg_dict = {key: 'last' for key in aggreagte_cols}

aggregated_df = training_df.groupby('STUDENT_ID').agg(last_agg_dict).reset_index(drop=True)

In [16]:
# Merging Pivot Columns into a dataframe
training_df = pd.concat([attendance_pivot.iloc[:, 1:], absent_pivot.iloc[:, 1:], present_pivot.iloc[:, 1:], enrolled_pivot.iloc[:, 1:], grade_pivot.iloc[:, 1:], imputed_pivot.iloc[:, 1:], aggregated_df], axis=1)

In [17]:
def build_features(df,
                   chronic_thr=0.95,
                   disadv_codes={1},
                   female_code="Female",
                   grade_map={-2:-2,-1:-1,0:0},
                ):

    yr_pat = re.compile(r"Attendance_Rate_(\d{4})$")
    years  = sorted(int(m.group(1)) for c in df.columns if (m := yr_pat.search(c)))

    for y in years:
        RATE      = f"Attendance_Rate_{y}"
        PRESENT   = f"Total_Days_Present_{y}"
        ENROLLED  = f"Total_Days_Enrolled_{y}"
        UNEXCUSED = f"Total_Days_Unexcused_Absent_{y}"
        GRADE_RAW = f"STUDENT_GRADE_LEVEL_{y}"

        days_enrolled = df[ENROLLED]
        df[f"Unexcused_Rate_{y}"] = df[UNEXCUSED] / days_enrolled
        df[f"Excused_Rate_{y}"] = 1.0 - df[RATE] - df[f"Unexcused_Rate_{y}"]
        df[f"Total_Absent_Days_{y}"] = df[ENROLLED] - df[PRESENT]
        df[f"Chronic_Absent_Flag_{y}"] = (df[RATE] < chronic_thr).astype("int8")

        prev1 = f"Attendance_Rate_{y-1}"
        prev2 = f"Attendance_Rate_{y-2}"
        df[f"Att_Rate_Prev1_{y}"] = df.get(prev1)
        df[f"Att_Rate_Prev2_{y}"] = df.get(prev2)
        df[f"Delta_Attendance_{y}"] = df[RATE] - df[f"Att_Rate_Prev1_{y}"]

        mean_cols = [c for c in (RATE, prev1, prev2) if c in df.columns]
        df[f"Att_Rate_Mean_3yr_{y}"] = df[mean_cols].mean(axis=1)
        df[f"Att_Rate_Std_Last3_{y}"] = df[mean_cols].std(axis=1, ddof=1) # Sample Standard Deviation

        abs_cols  = [f"Total_Absent_Days_{yr}" for yr in (y, y-1, y-2) if f"Total_Absent_Days_{yr}" in df.columns]
        df[f"Max_Absent_Days_Last3_{y}"] = df[abs_cols].max(axis=1)

        df[f"Grade_Numeric_{y}"] = df[GRADE_RAW].replace(grade_map).astype(float)
        prev_g = f"STUDENT_GRADE_LEVEL_{y-1}"
        df[f"Grade_Repeat_Flag_{y}"] = (df[GRADE_RAW] == df[prev_g]).astype("int8") if prev_g in df.columns else np.nan

        if y == years[0]:
            gcols = [f"STUDENT_GRADE_LEVEL_{yr}" for yr in years if f"STUDENT_GRADE_LEVEL_{yr}" in df.columns]
            df["_First_Grade_Num"] = df[gcols].bfill(axis=1).iloc[:, 0].astype(float)
            df["_First_Year"] = df[gcols].notna().idxmax(axis=1).str[-4:].astype(int)

        df[f"Grade_Gap_{y}"] = df[f"Grade_Numeric_{y}"] - (df["_First_Grade_Num"] + (y - df["_First_Year"]))

        grp = df.groupby("LOCATION_ID")[RATE]
        s_tot, s_cnt = grp.transform("sum"), grp.transform("count")
        df[f"School_Year_Avg_Att_{y}"] = np.where(s_cnt > 1, (s_tot - df[RATE]) / (s_cnt - 1), np.nan)

        grp = df.groupby("DISTRICT_CODE")[RATE]
        d_tot, d_cnt = grp.transform("sum"), grp.transform("count")
        df[f"District_Year_Avg_Att_{y}"] = np.where(d_cnt > 1, (d_tot - df[RATE]) / (d_cnt - 1), np.nan)

        gnum = f"Grade_Numeric_{y}"
        if gnum in df.columns:
            grp = df.groupby(["LOCATION_ID", gnum])[RATE]
            sg_tot, sg_cnt = grp.transform("sum"), grp.transform("count")
            df[f"School_Grade_Avg_Att_{y}"] = np.where(sg_cnt > 1, (sg_tot - df[RATE]) / (sg_cnt - 1), np.nan)

        df[f"School_Year_Rank_{y}"] = df.groupby("LOCATION_ID")[RATE].transform(lambda s: s.rank(method="average", pct=True))
        df[f"District_Year_Rank_{y}"] = df.groupby("DISTRICT_CODE")[RATE].transform(lambda s: s.rank(method="average", pct=True))
        df[f"Grade_Year_Rank_{y}"] = df.groupby(gnum)[RATE].transform(lambda s: s.rank(method="average", pct=True)) if gnum in df.columns else np.nan

        if gnum in df.columns:
            df[f"School_Grade_Year_Rank_{y}"] = df.groupby(["LOCATION_ID", gnum])[RATE].transform(lambda s: s.rank(method="average", pct=True))

        df[f"Grade_X_Gender_{y}"] = df[gnum].fillna(-99).astype(int).astype(str) + "_" + df["STUDENT_GENDER"].fillna("UNK")
        band = pd.cut(df[gnum], bins=[-np.inf, 5, 8, np.inf], labels=["K-5", "6-8", "9-12"])
        df[f"Econ_X_Grade_Band_{y}"] = band.astype(str) + "_" + df["ECONOMIC_CODE"].astype(str)

        active = df[ENROLLED] > 0
        schl = df[active].groupby("LOCATION_ID")
        df.loc[active, f"Pct_Econ_Disadv_School_{y}"] = schl["ECONOMIC_CODE"].transform(lambda s: np.isin(s, list(disadv_codes)).mean())
        df.loc[active, f"Pct_Female_School_{y}"] = schl["STUDENT_GENDER"].transform(lambda s: (s == female_code).mean())

        if gnum in df.columns:
            sg = df[active].groupby(["LOCATION_ID", gnum])
            df.loc[active, f"Pct_Econ_Disadv_School_Grade_{y}"] = sg["ECONOMIC_CODE"].transform(lambda s: np.isin(s, list(disadv_codes)).mean())
            df.loc[active, f"Pct_Female_School_Grade_{y}"] = sg["STUDENT_GENDER"].transform(lambda s: (s == female_code).mean())

        grp = df.groupby("ETHNIC_CODE")[RATE]
        e_tot, e_cnt = grp.transform("sum"), grp.transform("count")
        df[f"Ethnic_Year_Avg_Att_{y}"] = np.where(e_cnt > 1, (e_tot - df[RATE]) / (e_cnt - 1), np.nan)


    df.drop(columns=["_First_Grade_Num", "_First_Year"], inplace=True)
    return df


In [18]:
training_df = build_features(training_df)

/var/folders/g7/qvkgm1k12kzfvx8hv5_jqcph0000gn/T/ipykernel_59533/3261284008.py:35: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"Max_Absent_Days_Last3_{y}"] = df[abs_cols].max(axis=1)
/var/folders/g7/qvkgm1k12kzfvx8hv5_jqcph0000gn/T/ipykernel_59533/3261284008.py:37: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"Grade_Numeric_{y}"] = df[GRADE_RAW].replace(grade_map).astype(float)
/var/folders/g7/qvkgm1k12kzfvx8hv5_jqcph0000gn/T/ipykernel_59533/3261284008.py:39: PerformanceWarning: DataFrame is highly fragmented.  This i

In [20]:
training_df.to_parquet(config.PROCESSED_DATA_DIR / 'FinalTrainingData.parquet', index=False)